In [ ]:
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime
import re
import json
import pickle

In [ ]:
from IPython.display import display, Markdown

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
import httpimport

with httpimport.remote_repo(
    ['visualizador_dataframes', 'ferramentas_linguagem_natural'],
    'https://bonafe.github.io/CienciaDadosPython/src/componentes/'):
    
    from ferramentas_linguagem_natural import FerramentasLinguagemNatural
    
    from visualizador_dataframes import VisualizadorDataFrames    

In [ ]:
def prefixoData(data):
    return data.strftime("%Y_%m_%d")

In [ ]:
stopwords_conteudo_capacita = [
    "curso",
    "apresentação", 
    "turma", 
    "capacitação", 
    "aula", 
    "utilização", 
    "prático", 
    "estudo", 
    "treinamento", 
    "encontro", 
    "seminário",
    "especialização",
    "repasse",
    "funcionalidade",
    "sistema",
    "atuam",
    "atuar",
    "servidores",
    "utilização",
    "análise",
    "sobre",
    "objetivo",
    "área",
    "equipe",
    "visando",
    "habilitar",
    "conhecimento",
    "processo",
    "trabalho",
    "receita",
    "federal",
    "participante",
    "apresentar",
    "ações",
    "ação",
    "envolvendo",
    "desde",
    "rfb",
    "nbsp",
    "conhecer",
    "capacitar",
    "atualizar",
    "reciclar"
]

In [ ]:
df_instrutores_clusterizado = pd.read_csv("z:/bases_capacita/projeto_capacita/tcc_bonafe/2022_06_30-instrutores_clusterizado.csv", index_col=None, header=0)
df_instrutores_clusterizado.name = "instrutores"

In [ ]:
df_instrutores_clusterizado.info()

# Correlação entre os Clusters Originais

In [ ]:
df = df_instrutores_clusterizado.drop(columns = ['id_instrutoria', 'id_turma_execucao', 'id_acao', 'ano_conclusao',
       'ano_evento', 'data_inicio', 'data_fim', 'carga_horaria',
       'objetivo_projeto', 'tipo', 'tipo_competencia', 'titulo_competencia',
       'titulo_acao', 'conteudo_programatico',
       'nome_instrutor_anonimizado', 'nome_rcd_anonimizado', 'cluster_instrutor'])

In [ ]:
df = pd.get_dummies(df, prefix="cluster_conteudo_programatico", prefix_sep="_", dummy_na=False, columns=["cluster_conteudo_programatico"])
df = pd.get_dummies(df, prefix="cluster_titulo_acao", prefix_sep="_", dummy_na=False, columns=["cluster_titulo_acao"])
df = pd.get_dummies(df, prefix="cluster_objetivo_projeto", prefix_sep="_", dummy_na=False, columns=["cluster_objetivo_projeto"])
df = pd.get_dummies(df, prefix="cluster_nome_curso", prefix_sep="_", dummy_na=False, columns=["cluster_nome_curso"])

In [ ]:
df = df.groupby("matricula_siape_anonimizada").sum()

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

# Correlação entre os Clusters Padronizados

In [ ]:
df = df_instrutores_clusterizado.drop(columns = ['id_instrutoria', 'id_turma_execucao', 'id_acao', 'ano_conclusao',
       'ano_evento', 'data_inicio', 'data_fim', 'carga_horaria',
       'objetivo_projeto', 'tipo', 'tipo_competencia', 'titulo_competencia',
       'titulo_acao', 'conteudo_programatico',
       'nome_instrutor_anonimizado', 'nome_rcd_anonimizado',
       'cluster_conteudo_programatico', 'cluster_titulo_acao', 
        'cluster_nome_curso', 'cluster_objetivo_projeto', 'cluster_instrutor'])

In [ ]:
df = pd.get_dummies(df, prefix="cluster_conteudo_programatico_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_conteudo_programatico_padronizado"])
df = pd.get_dummies(df, prefix="cluster_titulo_acao_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_titulo_acao_padronizado"])
df = pd.get_dummies(df, prefix="cluster_nome_curso_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_nome_curso_padronizado"])
df = pd.get_dummies(df, prefix="cluster_objetivo_projeto_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_objetivo_projeto_padronizado"])

In [ ]:
df = df.groupby("matricula_siape_anonimizada").sum()

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

# Exibir as características de cada agrupamento de instrutores

In [ ]:
def exibirCaracteristicasClusterInstrutor(cluster_instrutor):
    
    df_cluster_instrutor = df_instrutores_clusterizado[df_instrutores_clusterizado['cluster_instrutor'] == cluster_instrutor]
    
    quantidade_registros = df_cluster_instrutor["id_instrutoria"].count()
    
    df_conteudo_programatico = df_cluster_instrutor.value_counts("cluster_conteudo_programatico_padronizado") \
        .rename_axis('categoria').to_frame("conteudo_programatico")
    
    df_titulo_acao = df_cluster_instrutor.value_counts("cluster_titulo_acao_padronizado") \
        .rename_axis('categoria').to_frame('titulo_acao')
    
    df_nome_curso = df_cluster_instrutor.value_counts("cluster_nome_curso_padronizado") \
        .rename_axis('categoria').to_frame('nome_curso')
    
    df_objetivo_projeto = df_cluster_instrutor.value_counts("cluster_objetivo_projeto_padronizado") \
        .rename_axis('categoria').to_frame('objetivo_projeto')
    
    df_total = pd.concat([df_conteudo_programatico, df_titulo_acao, df_nome_curso, df_objetivo_projeto], axis=1)
    
    df_total.sort_index(inplace=True)
    
    ax = df_total.plot.barh(figsize=(6,8), title=f'Cluster {cluster_instrutor} - {quantidade_registros} registros')
    
    ax.get_figure().patch.set_facecolor('white')
    

    display(df_cluster_instrutor.value_counts("matricula_siape_anonimizada").to_frame("instrutorias_por_instrutor").describe())
    

In [ ]:
exibirCaracteristicasClusterInstrutor(0)

In [ ]:
exibirCaracteristicasClusterInstrutor(1)

In [ ]:
exibirCaracteristicasClusterInstrutor(2)

In [ ]:
exibirCaracteristicasClusterInstrutor(3)

In [ ]:
exibirCaracteristicasClusterInstrutor(4)